# 🩻 Hyperparameter Tuning

This notebook is built for use on google colab.

## 1. Install Packages

Install the `ultralytics` package to access pre-trained YOLO models.

In [ ]:
!pip install ultralytics

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 67.4 MB/s eta 0:00:00


Install the `wandb` package for Weights and Biases website (https://wandb.ai/) integration.

In [ ]:
!pip install wandb

## 2. Import Necessary Packages

In [ ]:
from ultralytics import YOLO
import wandb
from wandb.integration.ultralytics import add_wandb_callback
import torch

from google.colab import drive
from google.colab import userdata
import zipfile
import os

Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.


wandb: WARNING This integration is tested and supported for ultralytics v8.0.238 and below.
wandb: WARNING             Please report any issues to https://github.com/wandb/wandb/issues with the tag `yolov8`.


## 3. Get Necessary Files

Loading the data from the google drive and unzipping.

In [ ]:
# Download the data from google drive
file_name = "/content/drive/MyDrive/Share Files/dvb_data_redux.zip"

In [ ]:
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# Extract zip file to the content/datasets directory
try:
    with zipfile.ZipFile(file_name, 'r') as zip_ref:
        zip_ref.extractall('/content/datasets/')
except FileNotFoundError:
    print(f"Error: The file {file_name} was not found.")
except zipfile.BadZipFile:
    print(f"Error: The file {file_name} is not a valid zip file.")
except Exception as e:
    print(f"An error occurred: {e}")

Create the YOLO model YAML file.

In [ ]:
# YOLO Model YAML file
yaml_filename = 'tuning.yaml'

yaml_content = """
path: dvb_data_redux  # dataset root dir (leave empty for HUB)
train: train  # train images (relative to 'path')
val:   val    # val images (relative to 'path')
test:  test   # test images (optional)

names:
  0: drone
"""

In [ ]:
# Write the YAML config file
model_path = "/content/model"

# Create the file


with open(os.path.join(model_path, yaml_filename), 'w') as f:
    f.write(yaml_content)

## 4. Model Tuning

Connect to wandb.

In [ ]:
KEY = userdata.get('WANDB_KEY')
wandb.login(key=KEY)

/usr/local/lib/python3.12/dist-packages/notebook/notebookapp.py:191: SyntaxWarning: invalid escape sequence '\/'
  | |_| | '_ \/ _` / _` |  _/ -_)
wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: amswanson333 (synth-drone) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


True

Load YOLO model.

In [ ]:
# Load pre-trained YOLO 11 model
model = YOLO('yolo11x.pt')

Define the hyperparameter search space. Defaults available here: https://docs.ultralytics.com/guides/hyperparameter-tuning/#default-search-space-description

In [ ]:
search_space = {
    "lr0": (1e-5, 1e-3),
    "lrf": (0.01, 1.0),
    "momentum": (0.8, 0.98),
    "weight_decay": (0.0, 1e-3),
    "hsv_h": (0.0, 0.1),
    "hsv_s": (0.0, 0.9),
    "hsv_v": (0.0, 0.9),
    "degrees": (0.0, 45.0),
    "translate": (0.0, 0.25),
    "shear": (0.0, 5.0),
    "perspective": (0.0, 0.001),
    "fliplr": (0.0, 0.5),
    "mosaic": (0.0, 1.0),
    "mixup": (0.0, 0.5)
}

Initialize wandb project.

In [ ]:
wandb.init(project="yolo-tuning")

Tune the model.

In [ ]:
# Verify CUDA support is enabled
if torch.cuda.is_available():
    print("CUDA is enabled. Training on GPU.")
    print("GPU Device:", torch.cuda.get_device_name(0))
else:
    print("CUDA is not available. Training on CPU.")

CUDA is enabled. Training on GPU.
GPU Device: NVIDIA A100-SXM4-80GB


In [ ]:
model.tune(
    data="/content/model/tuning.yaml",
    batch=0.85,
    epochs=20,
    patience=5,
    iterations=32,
    optimizer="AdamW",
    space=search_space
)

Tuner: Initialized Tuner instance with 'tune_dir=/content/runs/detect/tune'
Tuner: 💡 Learn about tuning at https://docs.ultralytics.com/guides/hyperparameter-tuning
Tuner: Starting iteration 1/32 with hyperparameters: {'lr0': 0.001, 'lrf': 0.01, 'momentum': 0.937, 'weight_decay': 0.0005, 'hsv_h': 0.015, 'hsv_s': 0.7, 'hsv_v': 0.4, 'degrees': 0.0, 'translate': 0.1, 'shear': 0.0, 'perspective': 0.0, 'fliplr': 0.5, 'mosaic': 1.0, 'mixup': 0.0}
Saved /content/runs/detect/tune/tune_scatter_plots.png
Saved /content/runs/detect/tune/tune_fitness.png

Tuner: 1/32 iterations complete ✅ (5620.84s)
Tuner: Results saved to /content/runs/detect/tune
Tuner: Best fitness=0.32952 observed at iteration 1
Tuner: Best fitness metrics are {'metrics/precision(B)': 0.86004, 'metrics/recall(B)': 0.63039, 'metrics/mAP50(B)': 0.69539, 'metrics/mAP50-95(B)': 0.32952, 'val/box_loss': 1.78477, 'val/cls_loss': 1.45, 'val/dfl_loss': 1.12768, 'fitness': 0.32952}
Tuner: Best fitness model is /content/runs/detect/trai

## 5. Model Training

Define the optimal parameters from the tuning.

In [ ]:
config = {
    "lr0": 0.00099,
    "lrf": 0.01081,
    "momentum": 0.85521,
    "weight_decay": 0.00052,
    "hsv_h": 0.01634,
    "hsv_s": 0.7638,
    "hsv_v": 0.38363,
    "translate": 0.07243,
    "fliplr": 0.5,
    "mosaic": 0.95755
}

Train the model.

In [ ]:
results = model.train(
    data='/content/model/tuning.yaml',
    batch=0.85,
    epochs=200,
    patience=20,
    optimizer="AdamW",
    device=0,
    **config
)

Ultralytics 8.3.204 🚀 Python-3.12.11 torch-2.8.0+cu126 CUDA:0 (NVIDIA A100-SXM4-40GB, 40507MiB)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=0.85, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, compile=False, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=/content/model/tuning.yaml, degrees=0.0, deterministic=True, device=0, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=200, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.01634, hsv_s=0.7638, hsv_v=0.38363, imgsz=640, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.00099, lrf=0.01081, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolo11x.pt, momentum=0.85521, mosaic=0.95755, multi_scale=False, name=train, nbs=64, nms=False, opset=None, optimize=False, optimizer=AdamW, overlap_mask=True, patience=20, per